# Processing Pasadena (Texas) jail logs: 8/18 - 5/21

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
import altair as alt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Read the raw data

The department provided the data in a [messy PDF format](data/raw/Restraint_Chair_Data.pdf). That limits this initial analysis in two ways: 1) some of the pages are scanned as images, rendering them flat (and impervious to PDF-to-text parsing software); and 2) the redactions in the PDF's narrative field make it impossible to extract in a machine readable format. We can talk about strategies for getting around this. It should also be noted that there are roughly 100 missing records because of the flat pages issue. Nevertheless, I was able to extract roughly 500 records from the PDF into a dataframe for analysis. 

In [4]:
src = pd.read_csv(
    "data/raw/chair-slices/tabula-chair-columns_0-6.csv", parse_dates=["date"]
)

### Dates

In [5]:
src["date"] = pd.to_datetime(src["date"], format="%m/%d/%Y")
src["date"] = pd.to_datetime(src["date"].dt.strftime("%Y-%m-%d"))
src["year"] = src["date"].dt.year
src["quarter"] = src["date"].dt.quarter
src["day"] = src["date"].dt.day
src["month"] = src["date"].dt.month
src["weekday"] = src["date"].dt.day_name()
src["monthname"] = src["date"].dt.month_name()
src["month_year"] = pd.to_datetime(src["date"]).dt.to_period("M")
src["month_year_full"] = src["date"].apply(lambda x: x.strftime("%B-%Y"))
src["month_year_full"] = pd.to_datetime(src["month_year_full"])

### Times

In [6]:
src["time"] = src["time"].astype(str).str.zfill(4)
src["hour"] = src["time"].str[:2]
src["minute"] = src["time"].str[-2:]
src["time_full"] = pd.to_datetime(
    src["hour"] + ":" + src["minute"], format="%H:%M"
).dt.time
src["hour"] = src["hour"].astype(int)
src["minute"] = src["minute"].astype(int)

In [7]:
src["location"] = src["location"].str.replace("\r", " ", regex=False)

In [8]:
src["narrative"] = ""

### How many records? 

In [9]:
len(src)

503

In [10]:
df = src.copy()

---

In [11]:
df.head()

,id,date,time,shift,reporting_personnel,number_assisting,location,year,quarter,day,month,weekday,monthname,month_year,month_year_full,hour,minute,time_full,narrative
0,201800110,2018-08-26,1445,EVES,"CORTES, A",3.0,AFIS ROOM,2018,3,26,8,Sunday,August,2018-08,2018-08-01,14,45,14:45:00,
1,201800112,2018-08-26,1445,EVES,"CORTES, A",3.0,CELL,2018,3,26,8,Sunday,August,2018-08,2018-08-01,14,45,14:45:00,
2,201800113,2018-08-26,1600,EVES,"CORTES, A",2.0,CELL,2018,3,26,8,Sunday,August,2018-08,2018-08-01,16,0,16:00:00,
3,201800114,2018-08-30,0315,NITE,"ADAME, R",5.0,SALLY PORT,2018,3,30,8,Thursday,August,2018-08,2018-08-01,3,15,03:15:00,
4,201800115,2018-09-01,0450,NITE,"BETANCOURT, L",2.0,SALLY PORT,2018,3,1,9,Saturday,September,2018-09,2018-09-01,4,50,04:50:00,


In [12]:
df.tail()

,id,date,time,shift,reporting_personnel,number_assisting,location,year,quarter,day,month,weekday,monthname,month_year,month_year_full,hour,minute,time_full,narrative
498,202100114,2021-04-20,0520,NITE,"BETANCOURT, L",3.0,CELL,2021,2,20,4,Tuesday,April,2021-04,2021-04-01,5,20,05:20:00,
499,202100116,2021-04-25,1000,DAYS,"MARROQUIN, J",3.0,BOOKING AREA,2021,2,25,4,Sunday,April,2021-04,2021-04-01,10,0,10:00:00,
500,202100121,2021-04-27,2335,NITE,"VILLEGAS, L",2.0,BOOKING AREA,2021,2,27,4,Tuesday,April,2021-04,2021-04-01,23,35,23:35:00,
501,202100124,2021-04-30,0100,NITE,"VILLEGAS, L",4.0,SALLY PORT,2021,2,30,4,Friday,April,2021-04,2021-04-01,1,0,01:00:00,
502,202100126,2021-05-05,2240,NITE,"KENNEDY, J",3.0,DWI ROOM,2021,2,5,5,Wednesday,May,2021-05,2021-05-01,22,40,22:40:00,


---

### How many restraint cases per year? 

In [13]:
years = df.groupby("year")["id"].size().reset_index()
years

,year,id
0,2018,88
1,2019,211
2,2020,152
3,2021,52


### Which weekdays had the most incidents? 

In [14]:
weekdays = df.groupby("weekday").size().reset_index(name="count")
weekdays.sort_values("count", ascending=False)

,weekday,count
2,Saturday,87
3,Sunday,85
6,Wednesday,76
0,Friday,74
5,Tuesday,71
4,Thursday,64
1,Monday,46


### At which hours were these incidents most common? 

In [15]:
hours = df.groupby("hour").size().reset_index(name="count")
hours.sort_values("count", ascending=False)

,hour,count
2,2,40
1,1,38
3,3,34
0,0,32
17,17,30
21,21,29
15,15,28
20,20,27
23,23,27
4,4,27


### Which locations were most common? 

In [16]:
locations = df.groupby("location").size().reset_index(name="count")
locations.sort_values("count", ascending=False)

,location,count
2,CELL,183
1,BOOKING AREA,152
5,SALLY PORT,113
0,AFIS ROOM,23
3,DWI ROOM,7
4,JUVENILE HOLDING AREA,3


### Who logged restraints the most (i.e., who to call)? 

In [17]:
personnel = df.groupby("reporting_personnel").size().reset_index(name="count")
personnel.sort_values("count", ascending=False).head(10)

,reporting_personnel,count
2,"BETANCOURT, L",72
12,"HO, T",42
18,"PINA, F",39
11,"HERRERA, M",37
15,"MARROQUIN, J",36
6,"CORTES, A",23
24,"THOMAS, R",22
29,"WALKER, K",21
17,"OROZCO, C",19
1,"ANTOS, M",18


### Which shifts were most common? 

In [18]:
shifts = df.groupby("shift").size().reset_index(name="count")
shifts.sort_values("count", ascending=False).head(10)

,shift,count
2,NITE,250
1,EVES,203
0,DAYS,50


### Incidents per day in 2020

In [19]:
df2020 = df[df["year"] == 2020].reset_index()

In [20]:
days2020 = df2020.groupby("date").size().reset_index(name="count")
days2020.sort_values("count", ascending=False).head(10)

,date,count
47,2020-06-10,4
41,2020-05-24,4
73,2020-09-08,3
65,2020-07-31,2
82,2020-09-30,2
80,2020-09-27,2
49,2020-06-17,2
96,2020-11-08,2
46,2020-06-09,2
83,2020-10-01,2


### How many times a week did this happen? 

In [21]:
(days2020["count"].sum() / 52).round(2)

2.92

---

### Cases in which the narrative mentions an "Officer Carter"

In [22]:
ids = (
    {
        "id": "201800034",
        "narrative": "While assigned to the jail position, the inmate BORJON,MARIO RAYMOND Booking#239273 was placed in the restraint chair at approximatley 2110hrs. I was notified by Supervisor Rhodes that the inmate was repeatedley strking the wall in cell 1705. In fear for his safety, we made our way to the inmate`s cell. Pso Teel, and Officer Carter assisted us in placing the inmate in the restraint chair with no further incident or injury. [05/31/2018 21:32, THO, PA , PPD]",
        "id": "201800048",
        "narrative": "At approximately 1815hrs, the inmate MEDINA,HUMBERTO Booking#240586 was placed in the restraint chair. Officer Carter requested for the beligerent inmate to be chaired upon arrival. PSO Betancourt assissted me in chairing the inmate with no further incident nor injury. Supervisor Rhodes was working MC and approved. [06/15/2018 18:46, THO, PA , PPD]",
        "id": "201800052",
        "narrative": "Prisoner Maestre,Armando Francisco was placed into the restraint chair upon arrival for being belligerent on scene. I PSO Betancourt walked out to the sally por to meet Ofc Carter with the prisoner. We placed the prisoner in the restraint chair and contined to place the prisoner into cell 1708 for further observation. Supervisor Degraff was notified of the incident [06/17/2018 18:01, LBETANCOURT,PA , PPD] [06/17/2018 18:30, LBETANCOURT, PA , PPD]",
        "id": "201800138",
        "narrative": "At aproximately 0030 prisoner Prince,Travis was brougth in to the jail. The prisoner constantly used verbally assaultive language towards officers and jail staff. The prisoner was padded down and placed in the holding cage where he began to hit the cage and said he was going to give the jail staff hell. The prisoner was advised to calm down and to take a seat and wait to be proccesed, but began to jump on the bench. Immediately Ofc Carter and myself PSO Betancourt walked in to the holding cage and told the prisoner to place his hands behind his back. Right away we took control of the prisoner holding his arms behind his back and walked him to the restraint chair. The prisoner was strapped down to the chair by Ofc Carter, PSO Pina, PSO Betancourt, with the prisoner giving some resistance and placed into cell 1708 to be further observed. The prisoner continued to use abusive language and threaten staff as they passed by the cell. PSO Walls was present in the jail when the incident took place, and there was no injuries. [09/24/2018 02:51, LBETANCOURT, PA , PPD]",
        "id": "201800143",
        "narrative": "At approximately 0240 prisoner Rychlik,Ambor began to kick the door complaining that it was cold and that we needed to give her a blanket. The prisoner was advised to stop kicking the door or she would be placed in to a restraint chair which she replied 'I dont give a fuck come and put me in one'! The prisoner contined to kick the door repeatedly. Immediately after PSO Pina, Ofc Carter, and myself PSO Betancourt made our way to cell G2. Upon arrival the prisoner was ordered to stand up, pull her arms out of her shirt and put her hands behind her back. The prisoner refused at first but complied after multiple commands. The prisoner was placed in the restraint chair while still using abusive language. Supervisor Adame was informed of the incident. [10/02/2018 03:22, LBETANCOURT, PA , PPD]",
        "id": "201800145",
        "narrative": "At aproximately 0510 prisoner Medellin,Jesus was placed in the restraint chair after several attempts to reason with the prisoner to calm down. The prisoner came in to the jail in a belligerent manner. The prisoner continously made agressive movements and threats towards officers and staff . The prisoner was placed in the restraint chair by myself PSO Betancourt, Ofc Carter, and PSO Valdez and placed in cell 1708. The prisoner continued to make threats and use assaultive langauage towards staff. PSO supervisor Adame was in the jail when the incident took place. [10/05/2018 05:39, LBETANCOURT, PA , PPD]",
        "id": "201900129",
        "narrative": "On this date at approximately 1830hrs the listed inmate was placed in the restraint chair for attempting to harm herself by hitting the door in her cell, playing in the toilet water, and threatening suicide. While on duty and working as the Jailer, I was notified by PSO Teel and Officer Carter that the inmate had stripped off all of her clothes and was hitting the cell door to her cell. During this time I attempted to diffuse the situation by talking to the inmate via the jail intercom system. The inmate insisted on continuing her belligerent behavior and began telling me over the intercom that she would commit suicide if we did not go into the cell and stop her. PSO`s Teel, Herrera, and I then entered cell 1705 and escorted the inmate to restraint chair cell 1707. The inmate was placed in the restraint chair in cell 1707 without further incident or injury. PSO Supervisor Rhodes was notified and approved. It should be noted that prior to entering cell 1705 the inmate had put her clothing back on and was adequately dressed. [05/02/2019 19:43, RTHOMAS, PA , PPD]",
        "id": "201900150",
        "narrative": "INMATE WAS CHAIRED UPON ARRIVAL DUE TO HIM BEING BELLIGERENT ON SCENE AND SPITTING AT OFFICER CARTERS BOOTS. [05/25/2019 15:16, MHERRERA, PA , PPD]",
        "id": "201900152",
        "narrative": "Inmate was checked out earlier by ems staff claiming to have extreme pain from spider bites and seizures. Inmate was throwing himself on the floor while bracing his fall and attempted to show signs of a seizure. Inmate was told this was not going to get him out of jail and he then began to beat on the cell door. Inmate was given a verbal warning to stop but again began to fall and catch himself and claim it was another seizure. Inmate was place in a restraint chair in cell by myself, Ofc Carter, and Ofc Perales. [05/29/2019 19:41, COROZCO, PA , PPD]",
        "id": "201900196",
        "narrative": "On this date at approximately 1510hrs the listed inmate was placed in the restraint chair for hitting his on the cage in the back of a patrol unit and resisting with officers and jail staff. While working the jailer position I was notified via the jail radio that Officer Carter requested the jail staff to have the restraint chair in the sally port when he arrived. Upon arrival I observed the inmate to be violently hitting his head on the cage inside of the vehicle in what appeared to be a deliberate attempt to hurt himself. Officer Carter and I then ordered the inmate out of the vehicle at which point her refused, the inmate also refused to walk once assisted out of the vehicle. Due to the inmates self harming and erratic behavior, I made the decision to place the inmate in the restraint chair for his safety and the safety of the officers and jail staff. While attempting to put the inmate in the restraint chair, he actively resisted and would not listen to verbal commands given. Once fully secured in the restraint chair and wheeled to cell 1708 there were no further incidents or injury. Patrol Supervisor Sgt King was on scene and approved of the restraining of the inmate and PSO Supervisor Rhodes was notified and approved as well. It should be noted that the inmate is continually threatening jail staff and making derogatory sexual remarks towards jail staff. [07/03/2019 16:48, RTHOMAS, PA , PPD]",
        "id": "201900373",
        "narrative": "On this day at approximately 1615hrs the listed inmate was placed in the restraint chair for attempting to harm himself and resisting jail staff. While working the jailer position I was notified that a belligerent inmate was enroute to the jail and he was attempting to harm himself and making threats to fight officers. I responded to the sally port with the restraint chair and contact officer Carter. Officer Carter asked the inmate to exit the vehicle and then had to assist him out. Once out of the vehicle the inmate was placed in the restraint chair and began actively resisting and attempted to spit on officers and jail staff. Once fully secured in the restraint chair he was taken to restraint chair cell 1707 without further incident or injury. Later in the day the inmate was approaching his time limit in the restraint chair, the inmate began to curse at jail staff when asked basic questions about his behavior. Furthermore the inmate threatened to assault jail staff and officers and began rocking the restraint chair in a violent manner in an attempt to flip it to harm himself. Patrol Sgt Lebedzinski was notified shortly after the inmate was placed in the restraint chair, and approved of the chairing event. [12/29/2019 20:48, RTHOMAS, PA , PPD]",
        "id": "202000040",
        "narrative": "At approximately 2210 prisoner Meehan,Brandon Lee DOB was placed in the wrap after refusing to stand up and come out of the holding cell. The prisoner became belligerent while also using profanity and pulling away when arresting Ofc Carter attempted to talk to the prisoner and assist the prisoner off the floor. I assisted Ofc Carter in getting the prisoner off the floor when the prisoner began pulling away from Ofc Carter. At this point Ofc Carter and my self placed the prisoner`s arms behind his back and requested the wrap system. We placed the prisoner in the wrap system with the help of PSO Walker and PSO Jalil. After being placed in cell 1708 Meehan began complaining of having high blood pressure problems, mental health problems such as bipolar disorder and being schizophrenic and then an hour and a half after being in the wrap the priosoner started to complain about having pains in his stomach due to a previous herina which he said he had already been having pains before coming to the jail. When i came back to the speak to the prisoner on a check shortly after the prisoner admitted that he just wanted to get out of jail and that he could sign a release for medical in order to get released. I explained to the prisoner that if he was seen by medical and needed to go to the hospital he would be released to go but would not be getting released to go home straight from the jail. When EMS did a vitals check on the prisoner they said everything looked normal and there was no need to worry especially about the blood pressure being too high. Meehan insisted in going because of pain in stomach due to the herina. Prisoner Meehan,Brandon was transported to the hospital and released from the jail with a rojo. [02/11/2020 01:38, LBETANCOURT, PA , PPD]",
        "id": "202000321",
        "narrative": "At approximately 1830, inmate Jackson, Keenan DOB was brought to jail by officer J. Carter. During the searching process inmate began to not follow directions given by officer Carter and myself. Inmate then told jail staff if we took the handcuffs off to search him he would not co operate with jail staff orders and even attempt to fight jail staff. Officer Carter and myself attempted to reason with Mr. Jackson multiple times to co operate with our booking and searching procedures and to make his stay with us as easy as possible. Mr. Jackson continued to raise his voice at jail staff and promising that he would fight if he was removed from handcuffs. Jail staff then placed Mr. Jackson into the restraint chair with little resistance. No further conflict occurred. [11/11/2020 18:54, GGONZALEZ, PAY , PPD]",
        "id": "202100012",
        "narrative": "Inmate was placed into restraint chair upon arrival by myself, PSO Cortes and Ofc Carter. Due to threatening to kill officers while en route to the jail and making threats to harm staff [01/19/2021 20:34, COROZCO, PA , PPD]",
        "id": "202100074",
        "narrative": "Inmate pushed button notifying jailers that he was suicidal. Inmate was then placed into the restraint chair by PSO Kidd and PSO Teel. Inmate was put into chair with no further incident and Sgt Hibbs was notified. [03/15/2021 18:05, JKIDD, PAY , PPD] After inmate was in chair for approximately 10 minutes PSO Teel made the decision to move the inmate to the wrap. The inmate was then moved to the wrap by PSO Kidd, PSO Teel, and OFC Carter. [03/15/2021 18:23, JKIDD, PAY , PPD] Inmate was moved to the wrap due to him continuing to try and injure himself in the restraint chair. [03/15/2021 18:45, JKIDD, PAY , PPD]",
        "id": "202100098",
        "narrative": "While PSO Clarke was about to take an inmate out of Cell 1733 , PSO Clarke heard a thump on the ground to which it sounded like someone was being restrained. I quickly closed the cell door and assisted Officer Slight with the inmate along with Officer Carter. The inmate had pulled away from Officer Slight when he was being brought into the booking area for a secondary search. PSO Betancourt left the booking office to get the restraint chair out of 1708. PSO Clarke,PSO Betancourt, and Officer Slight picked up the inmate and the inmate was properly restrained in the restraint chair. No issues or injuries occured during this incident of restraining the inmate. [04/03/2021 03:53, TPCLARKE, PAY , PPD]",
    },
)